In [59]:
import pandas as pd
from tqdm import tqdm
import requests
from sqlite3 import connect
import time
from concurrent.futures import ThreadPoolExecutor

In [60]:
# Load data

shots = pd.read_csv('./shots/2024_shots_g.csv', encoding='latin-1')
goals = pd.read_csv('20232024_goals.csv')

In [61]:
print(len(shots))
shots = shots[shots["goalie"] != "ENG"]
print(len(shots))

57413
57413


In [62]:
shots['team_skaters'] = shots['team_skaters'].astype(str)
shots['opposing_skaters'] = shots['opposing_skaters'].astype(str)

In [63]:
def split_skaters(skaters):
    skaters = skaters.split(',')
    return [x.strip(' ') for x in skaters]

shots['opposing_skaters'] = shots.apply(lambda row: split_skaters(row['opposing_skaters']), axis=1)
shots['team_skaters'] = shots.apply(lambda row: split_skaters(row['team_skaters']), axis=1)

shots['team_skaters']

0        [8478178, 8474564, 8476453, 8477992, 8479410, ...
1        [8478178, 8474564, 8476453, 8477992, 8479410, ...
2        [8484325, 8477992, 8475177, 8479661, 8478416, ...
3        [8479591, 8477992, 8477839, 8477426, 8480246, ...
4        [8475197, 8478468, 8478508, 8482062, 8477424, ...
                               ...                        
57408    [8482667, 8480172, 8479983, 8479316, 8480821, ...
57409    [8483490, 8481517, 8476434, 8480184, 8482745, ...
57410    [8473986, 8483490, 8475462, 8479368, 8474641, ...
57411    [8473986, 8483490, 8475462, 8479368, 8476434, ...
57412    [8480001, 8475842, 8483490, 8481517, 8476434, ...
Name: team_skaters, Length: 57413, dtype: object

In [64]:
# Create dictionaries for skater statistics
skaters = {}

for i in tqdm(range(0, len(shots))):
    shot = shots.iloc[i]
    if shot['strength'] != 'EV':
        continue
    # xGA
    for skater_id in shot['opposing_skaters']:
        skater_dict = skaters.get(skater_id, {'xGF': 0, 'xGA': 0, 'primary_assists_PP': 0, 'primary_assists_EV': 0, 'sh_xGA': 0})
        skater_dict['xGA'] += shot['xGF']
        skaters[skater_id] = skater_dict

    # xGF
    for skater_id in shot['team_skaters']:
        skater_dict = skaters.get(skater_id, {'xGF': 0, 'xGA': 0, 'primary_assists_PP': 0, 'primary_assists_EV': 0, 'sh_xGA': 0})
        skater_dict['xGF'] += shot['xGF']
        skaters[skater_id] = skater_dict

skaters

100%|██████████| 57413/57413 [00:03<00:00, 16823.82it/s]


{'8474568': {'xGF': 21.048641288700008,
  'xGA': 22.087591978,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0},
 '8474600': {'xGF': 54.57856918370002,
  'xGA': 50.14145186450001,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0},
 '8478508': {'xGF': 27.760162960300008,
  'xGA': 28.303003717699998,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0},
 '8482062': {'xGF': 26.67669861279999,
  'xGA': 24.358136177899986,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0},
 '8477424': {'xGF': 89.21037904810008,
  'xGA': 87.9391129343001,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0},
 '8476925': {'xGF': 29.22333463110003,
  'xGA': 29.353165977399982,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0},
 '8478178': {'xGF': 43.5449197856,
  'xGA': 43.72112968980004,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0},
 '8474564': {'xGF': 32.3474829148,
  'xGA': 40

In [65]:
# skaters.pop('nan')

In [66]:
limit = 100
start = 0
all_players = []

while True:
    url = f"https://api.nhle.com/stats/rest/en/skater/timeonice?isAggregate=false&isGame=false&sort=%5B%7B%22property%22:%22timeOnIce%22,%22direction%22:%22DESC%22%7D,%7B%22property%22:%22playerId%22,%22direction%22:%22ASC%22%7D%5D&start={start}&limit={limit}&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameTypeId=2%20and%20seasonId%3C=20232024%20and%20seasonId%3E=20232024"
    response = requests.get(url)
    data = response.json()

    # Check if there are more records to retrieve
    if not data["data"]:
        break

    # Add the current batch of players to the list
    all_players.extend(data["data"])

    # Increment the start parameter for the next request
    start += limit

In [67]:
all_players

[{'evTimeOnIce': 74388,
  'evTimeOnIcePerGame': 1199.80645,
  'gamesPlayed': 62,
  'lastName': 'Hughes',
  'otTimeOnIce': 1640,
  'otTimeOnIcePerOtGame': 136.66666,
  'playerId': 8480800,
  'positionCode': 'D',
  'ppTimeOnIce': 14953,
  'ppTimeOnIcePerGame': 241.17741,
  'seasonId': 20232024,
  'shTimeOnIce': 2774,
  'shTimeOnIcePerGame': 44.74193,
  'shifts': 1590,
  'shiftsPerGame': 25.64516,
  'shootsCatches': 'L',
  'skaterFullName': 'Quinn Hughes',
  'teamAbbrevs': 'VAN',
  'timeOnIce': 92115,
  'timeOnIcePerGame': 1485.7258,
  'timeOnIcePerShift': 57.93396},
 {'evTimeOnIce': 72029,
  'evTimeOnIcePerGame': 1220.8305,
  'gamesPlayed': 59,
  'lastName': 'Doughty',
  'otTimeOnIce': 1403,
  'otTimeOnIcePerOtGame': 100.21428,
  'playerId': 8474563,
  'positionCode': 'D',
  'ppTimeOnIce': 11709,
  'ppTimeOnIcePerGame': 198.45762,
  'seasonId': 20232024,
  'shTimeOnIce': 8354,
  'shTimeOnIcePerGame': 141.59322,
  'shifts': 1513,
  'shiftsPerGame': 25.64406,
  'shootsCatches': 'R',
  'ska

In [68]:
def update_stats(player_id):
    stats = skaters[player_id]
    
    while True:
        try:
            player = requests.get(f'https://api-web.nhle.com/v1/player/{player_id}/landing').json()
            break
        except (requests.exceptions.RequestException, ConnectionResetError) as err:
            time.sleep(10)
            print('Taking a break...')

    # Populate Dictionary with relevant data
    stats['firstName'] = player['firstName']["default"]
    stats['lastName'] = player['lastName']["default"]
    stats['birthDate'] = player['birthDate']
    stats['nationality'] = player['birthCountry']
    stats['position'] = player['position']

    if stats['position'] == 'G':
        return
    stats['height'] = player['heightInInches']
    stats['weight'] = player['weightInPounds']
    if "fullTeamName" not in player.keys():
        stats['team'] = "Chicago Blackhawks"
        print("Perry")
    else:
        stats['team'] = player["fullTeamName"]["default"]
    stats['handedness'] = player['shootsCatches']

    for player_ in all_players:
        if int(player_["playerId"]) == int(player_id):
            filtered_player = player_
            break
        else:
            filtered_player = None
    if filtered_player:
        toi_stats = filtered_player
    else:
        toi_stats = None


    # TOI
    stats['TOI'] = toi_stats['timeOnIcePerGame'] /60 
    stats['EV_TOI'] = toi_stats['evTimeOnIcePerGame'] /60
    stats['PP_TOI'] = toi_stats['ppTimeOnIcePerGame'] /60
    stats['SH_TOI'] = toi_stats['shTimeOnIcePerGame'] /60
    stats['TOTAL_TOI'] = toi_stats['timeOnIce'] /60
    stats['TOTAL_EV_TOI'] = toi_stats['evTimeOnIce'] /60
    stats['TOTAL_PP_TOI'] = toi_stats['ppTimeOnIce'] /60
    stats['TOTAL_SH_TOI'] = toi_stats['shTimeOnIce'] /60
    stats['games'] = toi_stats['gamesPlayed']


    stats['xGF'] = stats['xGF'] / stats['TOTAL_EV_TOI'] * 60 if stats['TOTAL_EV_TOI'] != 0 else 0
    stats['xGA'] = stats['xGA'] / stats['TOTAL_EV_TOI'] * 60 if stats['TOTAL_EV_TOI'] != 0 else 0

    # Rate statistics
    nhl_seasons = list(filter(lambda x: x['leagueAbbrev'] == 'NHL' and x["season"] == 20232024, player['seasonTotals']))
    goals = 0
    sh_goals = 0
    pp_goals = 0
    pp_points = 0
    for seas in nhl_seasons:
        goals += seas["goals"]
        sh_goals += seas["shorthandedGoals"]
        pp_goals += seas["powerPlayGoals"]
        pp_points += seas["powerPlayPoints"]
    
    stats['goals_60'] = (goals - sh_goals - pp_goals) / stats['TOTAL_EV_TOI'] * 60 if stats['TOTAL_EV_TOI'] != 0 else 0
    stats['pp_60'] = pp_points / stats['TOTAL_PP_TOI'] * 60 if stats['TOTAL_PP_TOI'] != 0 else 0

    skaters[player_id] = stats

with ThreadPoolExecutor(max_workers=20) as executor:
    list(tqdm(executor.map(update_stats, skaters.keys()), total=len(skaters)))

 25%|██▌       | 242/960 [00:03<00:11, 64.28it/s]

Perry


 36%|███▌      | 344/960 [00:05<00:08, 69.32it/s]

Perry


100%|██████████| 960/960 [00:22<00:00, 43.49it/s]


In [69]:
# # Convert Dict to Pandas

# players = []

# for player_id, stats in skaters.items():
#     if stats['position'] == 'G':
#         continue
#     stats['player_id'] = str(player_id).rstrip('0').rstrip('.')[0:7]
#     splits  = player_splits[player_id]
#     players.append(pd.DataFrame.from_dict([splits]))

# df = pd.concat(players)
# print(df.columns)
# #df.to_sql('splits', conn, if_exists='replace')

In [70]:
# Shorthanded Rate

for i in tqdm(range(0, len(shots))):
    shot = shots.iloc[i]
    if shot['strength'] != 'Shorthanded':
        continue
    # xGA
    for skater_id in shot['opposing_skaters']:
        skater_dict = skaters.get(skater_id)
        if skater_dict == None:
            continue
        skater_dict['sh_xGA'] += shot['xGF']
        skaters[skater_id] = skater_dict

for player_id, stats in skaters.items():
    if stats['position'] == 'G':
        continue
    if 'TOTAL_SH_TOI' not in stats.keys():
        stats['sh_xGA'] = 0
        continue
    stats['sh_xGA'] = stats['sh_xGA'] / stats['TOTAL_SH_TOI']if stats['TOTAL_SH_TOI'] != 0 else 0
    stats['sh_xGA'] *= 60
    skaters[player_id] = stats

skaters

100%|██████████| 57413/57413 [00:02<00:00, 25856.58it/s]


{'8474568': {'xGF': 2.2313704360941142,
  'xGA': 2.3415097947760533,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 0.06006134761544284,
  'firstName': 'Luke',
  'lastName': 'Schenn',
  'birthDate': '1989-11-02',
  'nationality': 'CAN',
  'position': 'D',
  'height': 74,
  'weight': 225,
  'team': 'Nashville Predators',
  'handedness': 'R',
  'TOI': 15.091269833333333,
  'EV_TOI': 13.4757935,
  'PP_TOI': 0.042857,
  'SH_TOI': 1.572619,
  'TOTAL_TOI': 633.8333333333334,
  'TOTAL_EV_TOI': 565.9833333333333,
  'TOTAL_PP_TOI': 1.8,
  'TOTAL_SH_TOI': 66.05,
  'games': 42,
  'goals_60': 0.10601018875703053,
  'pp_60': 0.0},
 '8474600': {'xGF': 2.6814811401222816,
  'xGA': 2.4634826365723184,
  'primary_assists_PP': 0,
  'primary_assists_EV': 0,
  'sh_xGA': 6.99461789582581,
  'firstName': 'Roman',
  'lastName': 'Josi',
  'birthDate': '1990-06-01',
  'nationality': 'SUI',
  'position': 'D',
  'height': 73,
  'weight': 201,
  'team': 'Nashville Predators',
  'handedness': 'L

In [71]:
# Primary Assist Contribution
for i in tqdm(range(0, len(goals))):
    if str(goals.iloc[i]['Assist 1']) != 'nan':
        player_id = str(goals.iloc[i]['Assist 1'])[:7]
        if player_id == '8484720' or player_id =='8482018':
            continue
        stats = skaters[player_id]
        if stats['position'] == 'G':
            continue
        if goals.iloc[i]['strength'] == 'EV':
            stats['primary_assists_EV'] += 1
        elif goals.iloc[i]['strength'] == 'Powerplay':
            stats['primary_assists_PP'] += 1
        
        skaters[player_id] = stats
    
for player_id, stats in tqdm(skaters.items()):
    if stats['position'] == 'G':
        continue
    if 'TOTAL_EV_TOI' not in stats.keys():
        stats['primary_assists_EV'] = 0
        stats['primary_assists_PP'] = 0
        continue
    stats['primary_assists_EV'] = stats['primary_assists_EV'] / (stats['TOTAL_EV_TOI']) if stats['TOTAL_EV_TOI'] != 0 else 0
    stats['primary_assists_EV'] *= 60

    stats['primary_assists_PP'] = stats['primary_assists_PP'] /(stats['TOTAL_PP_TOI']) if stats['TOTAL_PP_TOI'] != 0 else 0
    stats['primary_assists_PP'] *= 60

    skaters[player_id] = stats

100%|██████████| 960/960 [00:00<00:00, 932499.27it/s]


In [72]:
# Get Shots Per Player
account_id = 6415718365001
shots_goals = shots[shots['isGoal'] == 1]
for i in tqdm(range(0, len(shots_goals))):
    shot = shots_goals.iloc[i]

    stats = skaters[str(int(shot['shooter_id']))[:7]]

    if stats['position'] == 'G':
        continue

    player_shots = stats.get('shots', [])

    event_id = str(shot['shot_id'])
    game = shot['gameID']
    link = ['No link found.']

    # Grab media (shoutout trusty)
    if event_id != 'nan':
        failed = True
        while failed:
            try:
                media = requests.get(f'https://forge-dapi.d3.nhle.com/v2/content/en-US/videos?context.slug=nhl&tags.slug=highlight&tags.slug=gameid-{game}').json()
                failed = False
            except (requests.exceptions.RequestException,
                ConnectionResetError) as err:
                time.sleep(20)
                print('Taking a break...')
        highlights =  media["items"]
        shot_list = list(filter(lambda x: "fields" in x.keys(), highlights))
        shot_list = list(map(lambda x: x["fields"], shot_list))
        shot_list = list(filter(lambda x: "statsEventId" in x.keys() and x["statsEventId"] == event_id, shot_list))
        if len(shot_list) != 0:
            shot_id = shot_list[0]['statsEventId']
            if str(event_id) == shot_id:
                link = f'https://players.brightcove.net/6415718365001/EXtG1xJ7H_default/index.html?videoId={shot_list[0]["brightcoveId"]}'

    shooter = shot['shooter']
    if shooter == "K'Andre Miller":
        print('Unserious')
        shooter = "KAndre Miller"

    player_shots.append({'date': shot['date'], 'shooter': shooter, 'goalie': shot['goalie'], 'strength': shot['strength'], 'away_goals': shot['away_goals'], 'home_goals': shot['home_goals'], 'home_team': shot['home'], 'away_team': shot['away'], 'x': shot['x'], 'y': shot['y'], 'prev_event': shot['prev_event'], 'type': shot['shotType'], 'xG': shot['xGF'], 'Outcome': shot['event'], 'Link': link})

    stats['shots'] = player_shots
    skaters[shot['shooter_id']] = stats

  0%|          | 0/5486 [00:00<?, ?it/s]

 13%|█▎        | 708/5486 [01:28<09:20,  8.52it/s]

Unserious


 33%|███▎      | 1816/5486 [03:41<06:51,  8.91it/s]

Unserious


...

 56%|█████▋    | 3091/5486 [06:19<05:19,  7.50it/s]

Unserious


 56%|█████▋    | 3094/5486 [06:20<04:53,  8.16it/s]

Unserious


100%|██████████| 5486/5486 [11:18<00:00,  8.08it/s]


In [73]:
# Get Assists Per Player
shots_goals = shots[shots['isGoal'] == 1]
for i in tqdm(range(0, len(shots_goals))):
    shot = shots_goals.iloc[i]
    if str(shot['primary_assist']) == 'nan':
        continue
    stats = skaters[str(int(shot['primary_assist']))[:7]]

    if stats['position'] == 'G':
        continue

    player_shots = stats.get('assists', [])

    event_id = str(shot['shot_id'])
    game = shot['gameID']
    link = ['No link found.']

    # Grab media (shoutout trusty)
    if event_id != 'nan':
        failed = True
        while failed:
            try:
                media = requests.get(f'https://forge-dapi.d3.nhle.com/v2/content/en-US/videos?context.slug=nhl&tags.slug=highlight&tags.slug=gameid-{game}').json()
                failed = False
            except (requests.exceptions.RequestException,
                ConnectionResetError) as err:
                time.sleep(20)
                print('Taking a break...')
        highlights =  media["items"]
        shot_list = list(filter(lambda x: "fields" in x.keys(), highlights))
        shot_list = list(map(lambda x: x["fields"], shot_list))
        shot_list = list(filter(lambda x: "statsEventId" in x.keys() and x["statsEventId"] == event_id, shot_list))
        if len(shot_list) != 0:
            shot_id = shot_list[0]['statsEventId']
            if str(event_id) == shot_id:
                link = f'https://players.brightcove.net/6415718365001/EXtG1xJ7H_default/index.html?videoId={shot_list[0]["brightcoveId"]}'

    shooter = shot['shooter']
    if shooter == "K'Andre Miller":
        print('Unserious')
        shooter = "KAndre Miller"

    


    player_shots.append({'date': shot['date'], 'shooter': shooter, 'goalie': shot['goalie'], 'strength': shot['strength'], 'away_goals': shot['away_goals'], 'home_goals': shot['home_goals'], 'home_team': shot['home'], 'away_team': shot['away'], 'x': shot['x'], 'y': shot['y'], 'prev_event': shot['prev_event'], 'type': shot['shotType'], 'xG': shot['xGF'], 'Outcome': shot['event'], 'Link': link})

    stats['assists'] = player_shots
    skaters[shot['primary_assist']] = stats

  0%|          | 0/5486 [00:00<?, ?it/s]

 13%|█▎        | 707/5486 [01:23<07:20, 10.85it/s]

Unserious


 33%|███▎      | 1816/5486 [03:34<06:19,  9.67it/s]

Unserious


...

 56%|█████▋    | 3091/5486 [06:04<04:32,  8.79it/s]

Unserious


 56%|█████▋    | 3093/5486 [06:05<04:18,  9.25it/s]

Unserious


100%|██████████| 5486/5486 [10:47<00:00,  8.47it/s]


In [74]:
# Goals Scored Above Expected
for player_id, stats in tqdm(skaters.items()):
    xG = sum(shots[shots['shooter_id'] == player_id]['xGF'])
    goals = sum(shots[shots['shooter_id'] == player_id]['isGoal'])

    stats['gsae'] = goals - xG

100%|██████████| 1710/1710 [00:00<00:00, 2485.19it/s]


In [75]:
# Convert Dict to Pandas

players = []

for player_id, stats in skaters.items():
    if player_id == "nan":
        continue
    str_id = str(int(player_id))[:7]
    if len(str_id) != 7:
        continue
    stats['player_id'] = str_id
    players.append(pd.DataFrame.from_dict([stats]))

df = pd.concat(players)
    

In [76]:
conn = connect("player_stats.db")
curr = conn.cursor()

# create a table 
curr.execute('CREATE TABLE IF NOT EXISTS players_2024 (xGF NUMBER, xGA NUMBER, primary_assists_pp NUMBER, primary_assists_EV NUMBER, sh_xGA NUMBER, firstName TEXT, lastName TEXT, birthDate TEXT, nationality TEXT, position TEXT, height TEXT, weight NUMBER, team TEXT, handedness TEXT, TOI NUMBER, EV_TOI NUMBER, PP_TOI NUMBER, SH_TOI NUMBER, goals_60 NUMBER, pp_60 NUMBER, shots TEXT, assists TEXT, gsae NUMBER, player_id TEXT)')
 
# commit the query
conn.commit()

In [77]:
df.dtypes

xGF                   float64
xGA                   float64
primary_assists_PP    float64
primary_assists_EV    float64
sh_xGA                float64
firstName              object
lastName               object
birthDate              object
nationality            object
position               object
height                float64
weight                float64
team                   object
handedness             object
TOI                   float64
EV_TOI                float64
PP_TOI                float64
SH_TOI                float64
TOTAL_TOI             float64
TOTAL_EV_TOI          float64
TOTAL_PP_TOI          float64
TOTAL_SH_TOI          float64
games                 float64
goals_60              float64
pp_60                 float64
shots                  object
assists                object
gsae                  float64
player_id              object
dtype: object

In [78]:
# Make sure types match up
df['firstName'] = df['firstName'].astype(str)
df['lastName'] = df['lastName'].astype(str)
df['birthDate'] = df['birthDate'].astype(str)
df['nationality'] = df['nationality'].astype(str)
df['height'] = df['height'].astype(str)
df['team'] = df['team'].astype(str)
df['handedness'] = df['handedness'].astype(str)
df['player_id'] = df['player_id'].astype(str)
df['shots'] = df['shots'].astype(str)
df['assists'] = df['assists'].astype(str)

In [79]:
# Ice Time Minimums
df = df[df['TOTAL_TOI'] >= 200]
df.drop(['TOTAL_TOI', 'games', 'TOTAL_EV_TOI', 'TOTAL_PP_TOI', 'TOTAL_SH_TOI'], inplace=True, axis=1)

In [80]:
conn.close()
conn = connect("player_stats.db")
curr = conn.cursor()
curr.execute('DROP TABLE IF EXISTS players_no_percentile_2024')
# create a table 
curr.execute('CREATE TABLE IF NOT EXISTS players_no_percentile_2024 (xGF NUMBER, xGA NUMBER, primary_assists_pp NUMBER, primary_assists_EV NUMBER, sh_xGA NUMBER, firstName TEXT, lastName TEXT, birthDate TEXT, nationality TEXT, position TEXT, height TEXT, weight NUMBER, team TEXT, handedness TEXT, TOI NUMBER, EV_TOI NUMBER, PP_TOI NUMBER, SH_TOI NUMBER, goals_60 NUMBER, pp_60 NUMBER, shots TEXT, gsae NUMBER, player_id TEXT)')
 
# commit the query
conn.commit()
df.drop_duplicates(subset=['player_id'], inplace=True)
df.to_sql('players_no_percentile_2024', conn, if_exists='replace')

685

In [81]:
conn.close()

In [82]:
# Split players by forward and defensemen
forwards = df[df['position'] != 'D']
df = df[df['position'] == 'D']

In [83]:
# Convert statistics to percentile
forwards['xGF'] = forwards.xGF.rank(pct = True)
forwards['xGA'] = forwards.xGA.rank(pct = True, ascending = False)
forwards['primary_assists_PP'] = forwards.primary_assists_PP.rank(method= 'min', pct = True)
forwards['primary_assists_EV'] = forwards.primary_assists_EV.rank(method= 'min', pct = True)
forwards['sh_xGA'] = forwards.sort_values(['SH_TOI'], ascending=False)['sh_xGA'].rank(method= 'first', pct = True, ascending = False)
forwards['TOI'] = forwards.TOI.rank(pct = True)
forwards['EV_TOI'] = forwards.EV_TOI.rank(pct = True)
forwards['PP_TOI'] = forwards.PP_TOI.rank(method='min', pct = True)
forwards['SH_TOI'] = forwards.SH_TOI.rank(method='min', pct = True)
forwards['goals_60'] = forwards.goals_60.rank(method='min', pct = True)
forwards['pp_60'] = forwards.pp_60.rank(method= 'min', pct = True)
forwards['gsae'] = forwards.gsae.rank(pct = True)

df['xGF'] = df.xGF.rank(pct = True)
df['xGA'] = df.xGA.rank(pct = True, ascending = False)
df['primary_assists_PP'] = df.primary_assists_PP.rank(method= 'min', pct = True)
df['primary_assists_EV'] = df.primary_assists_EV.rank(method= 'min', pct = True)
df['sh_xGA'] = df.sort_values(['SH_TOI'], ascending=False)['sh_xGA'].rank(method= 'first', pct = True, ascending = False)
df['TOI'] = df.TOI.rank(pct = True)
df['EV_TOI'] = df.EV_TOI.rank(pct = True)
df['PP_TOI'] = df.PP_TOI.rank(method='min', pct = True)
df['SH_TOI'] = df.SH_TOI.rank(method='min', pct = True)
df['goals_60'] = df.goals_60.rank(method='min', pct = True)
df['pp_60'] = df.pp_60.rank(method= 'min', pct = True)
df['gsae'] = df.gsae.rank(pct = True)

In [84]:
# Combine

df = pd.concat([df, forwards], ignore_index=False)

In [86]:
# Dump to SQL
conn = connect("player_stats.db")
curr = conn.cursor()
df.to_sql('players_2024', conn, if_exists='replace')

685

In [87]:
conn.close()